In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rome
import transformers

DEVICE = 'cuda:0'

tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2")
model = transformers.AutoModelForCausalLM.from_pretrained("gpt2")
model.to(DEVICE)

In [3]:
len(model.transformer.h)

12

In [6]:
import rome
rome.find_most_activated_layer(tokenizer, model, model.transformer.h, rome.ALL_FACTS[0])

[[5308, 1030, 36962, 259, 2577], [26552, 268, 371, 7211], [32718, 8882, 2586], [34831, 44289], [8600, 7637, 494, 47169], [46079, 11719, 41946]]
[[26552, 268, 371, 7211], [8600, 7637, 494, 47169]]
tensor([[[ -29.8800,  -29.2174,  -32.1315,  ...,  -38.0570,  -37.5303,
           -30.1930],
         [ -82.1039,  -81.4765,  -87.1828,  ...,  -91.2751,  -86.8619,
           -83.8059],
         [ -63.1326,  -62.2446,  -69.0401,  ...,  -77.1777,  -70.7937,
           -64.0932],
         ...,
         [-242.4443, -241.3117, -248.5987,  ..., -258.2960, -262.1668,
          -242.8523],
         [-231.6764, -230.5102, -236.4398,  ..., -245.1952, -247.4976,
          -232.5190],
         [  18.7897,   21.8216,   20.7391,  ...,    9.3397,   10.0078,
            22.7519]],

        [[ -32.1771,  -31.4158,  -33.7565,  ...,  -40.4558,  -39.3682,
           -32.2841],
         [ -77.9841,  -76.7804,  -82.0229,  ...,  -85.6624,  -82.3194,
           -77.1929],
         [ -68.7296,  -69.5368,  -71.0616,  

IndexError: index 1 is out of bounds for dimension 0 with size 1